In [1]:
!pip install timesfm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.8 MB/s eta 0:00:00


In [19]:
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import timesfm

# (Optional) If you're working inside Jupyter
%load_ext autoreload
%autoreload 2
%matplotlib inline



 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.12 (main, Apr  9 2025, 08:55:54) [GCC 11.4.0].
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [56]:
from data_processor_v2 import DataProcessorUpdated

processor = DataProcessorUpdated(self_back_window = 8*24)


In [57]:
processor.load_and_clean_data()
processor.combine_all_data()
processor.shift_dap()
processor.split_data(feature_columns=['SCED_system_lambda'])
processor.standardize_data()
processor.shift_data()

2023-12-31 23:00:00


/content/data_processor_v2.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[time_col] = pd.to_datetime(df[time_col])


2024-01-01 23:00:00
2023-12-31 23:00:00
2023-12-31 23:00:00


In [58]:
df, x_train_lstm, y_train_lstm, x_val_lstm, y_val_lstm, x_test_lstm, y_test_lstm = processor.get_data()

In [59]:
x_test_lstm.shape

(8713, 192, 1)

In [60]:
x_test = np.squeeze(x_test_lstm, axis=-1)

In [61]:
len(x_test)

8713

In [63]:
# Loading the timesfm-2.0 checkpoint:
# For Torch
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=24,
          num_layers=50,
          use_positional_embedding=False,
          context_len=8*24,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [64]:
y_pred, _ = tfm.forecast(x_test)

In [65]:
y_pred_normalized = y_pred  # from tfm.forecast()
y_test_normalized = processor.y_test_lstm


In [67]:
# 1. Flatten both (remove batch and window dimensions if needed)
y_pred_flat = y_pred_normalized.reshape(-1)
y_test_flat = y_test_normalized.reshape(-1)

# 2. Denormalize both using your processor's function
y_pred_real = processor.denormalization(y_pred_flat)
y_test_real = processor.denormalization(y_test_flat)


In [70]:
from sklearn.metrics import mean_absolute_error
import numpy as np

mae = mean_absolute_error(y_test_real, y_pred_real)
mse = np.mean((y_test_real - y_pred_real) ** 2)
variance = np.var(y_test_real)
smse = mse / variance

print(f"MAE: {mae:.4f}, SMSE: {smse:.4f}")


MAE: 30.9732, SMSE: 0.9721
